<a href="https://colab.research.google.com/github/risguptaa/capstone-langchain-chatbot-starter/blob/main/Bhagwat_ChatGPT_WORK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Generative Question Answering






## Installation

1. Click File -> Save a Copy in Drive. This will create a copy for you to modify in your Google Drive.

2. This project will require you to install langchain, cohere, chromadb, and other libraries. Run the command below to start the installation. Installation may take a couple of minutes.


3. Website link:

https://www.gita-society.com/bhagavad-gita-in-english-source-file.pdf

In [ ]:

!pip install cohere==4.57 nltk==3.8 unstructured langchain==0.0.195 chromadb==0.3.26 > /dev/null 2>&1

In [ ]:
pip install cohere==4.57 nltk==3.8 chromadb==0.3.26

In [ ]:
!pip install  unstructured langchain==0.0.195

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.2/152.2 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 23.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of unstructured-client to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of unstructured-client to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 44.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 6.8 MB/s eta 0:00:00
  

In [ ]:
!pip install PyPDF2


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.8 MB/s eta 0:00:00


In [ ]:
!pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 4.4 MB/s eta 0:00:00


## Import Libraries




In [ ]:
import os
import nltk
nltk.download('punkt')

from langchain import Cohere
from langchain.chains import RetrievalQA
from langchain.document_loaders import DirectoryLoader
from langchain.embeddings import CohereEmbeddings
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


ModuleNotFoundError: No module named 'langchain_community'

In [ ]:
# fixes nltk path issue on colab
!cp -R ~/nltk_data/tokenizers/punkt/PY3 ~/nltk_data/tokenizers/punkt/PY3_tab

## Setup

Before running the next cell, make sure you have a [Cohere API key](https://dashboard.cohere.ai/api-keys). Enter your API key using getpass in the cell below.


In [ ]:
from getpass import getpass
api_key = getpass()
os.environ["COHERE_API_KEY"] = api_key

··········


In [ ]:
!curl -o bhagavad-gita.pdf https://www.gita-society.com/bhagavad-gita-in-english-source-file.pdf

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  527k  100  527k    0     0  1299k      0 --:--:-- --:--:-- --:--:-- 1299k


In [ ]:
#!curl -o /content/documents/quantumcomputers.txt https://raw.githubusercontent.com/Thinkful-Ed/ai-in-web-dev-resources/refs/heads/main/books/quantumcomputers.txt --create-dirs

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  115k  100  115k    0     0   449k      0 --:--:-- --:--:-- --:--:--  449k


## Load the documents

The [DirectoryLoader](https://js.langchain.com/docs/api/document_loaders_fs_directory/classes/DirectoryLoader) class will allow you to load multiple documents that are in a directory. Create a directory by clicking in the folder icon at the left and upload your text documents there. If you want to use another format, such as `md` files, make sure to change the `glob="**/*.txt"` variable and read the documentation to make sure that the file type is supported.

**Note:** Cohere is now imposing a limit on embeddings. If you get an error try a smaller text (like a wikipedia article).

In [ ]:
#/content/BhagavadGita.pdf
from langchain.document_loaders import DirectoryLoader, PyPDFLoader

# Configure the loader to load PDF files from the specified directory
pdf_loader = DirectoryLoader('/content/', glob="**/*.pdf", loader_cls=PyPDFLoader)

# Load the PDF documents
pdf_documents = pdf_loader.load()

# View the loaded documents
#print(f"Loaded {len(pdf_documents)} PDF documents.")
##   print(document.page_content)  # This prints the content of each PDF file


In [ ]:
  #loader = DirectoryLoader('/content/documents', glob="**/*.txt")

 # import all txt files in the documents folder
#documents = loader.load()
#print (f'You have {len(documents)} document(s) in your documents folder')

## Splitting Text

Splitting text is useful because when you have a long document, it can be difficult to find the specific information relevant to a question. The `CharacterTextSplitter` helps break down the text into smaller, more manageable chunks.

In [ ]:
#text_splitter = CharacterTextSplitter(chunk_size=1500)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=3000, chunk_overlap=200)
texts = text_splitter.split_documents(pdf_documents)
print (f'Your document(s) was/were splitted into {len(texts)} chunks.')

Your document(s) was/were splitted into 57 chunks.


In [ ]:
texts[21]

Document(page_content='16                           International Gita Society \n \n \n6. Path of meditation \nA Karma-yogi is also a renunciant \nThe Supreme Lord said: One who performs the prescribed duty \nwithout seeking its fruit only for personal enjoyment, is a renunciant \n(Samnyasi) and a Karma-yogi. One does not become Samny asi \nmerely by not lighting the fire, and one does not become a yogi \nmerely by abstaining from work. (6.01) O Arjuna, what they call \nrenunciation (Samnyasa) is also known as Karma-yoga. No one \nbecomes a Karma-yogi who has not renounced the selfish motive \nbehind an action. (6.02)  \nA definition of yoga and yogi \nFor the wise who seeks to attain yoga of meditation or calm-\nness of mind, Karma-yoga is said to be the means. For the one \nwho has attained yoga, the calmness becomes the means of Self-\nrealization. A person is said to have attained yogic perfection when  \none has no desire for sensual pleasures or attachment to the fruits \nof work

## Embeddings and RetrievalQA

In simple terms, this code sets up the necessary tools for question-answering. It converts text into numerical representations called embeddings, creates a search index for the documents, and sets up a retrieval-based question-answering system using a specific type of chain (RetrievalQA).


In [ ]:
embeddings = CohereEmbeddings(cohere_api_key=os.getenv("COHERE_API_KEY"))
docsearch = Chroma.from_documents(texts, embeddings, persist_directory = 'db')
docsearch.persist() # save db
qa = RetrievalQA.from_chain_type(llm=Cohere(), retriever=docsearch.as_retriever(), return_source_documents=True)

In [ ]:
query = "what is karma?"
result = qa({"query": query})
result['result']

unknown field: parameter model is not a valid field


" Karma is the eternal justice and the eternal law that one has control over their respective duty, but no control or claim over the results.  The fruits of work should not be one's only motive and they should never be inactive. One must learn to gladly endure all that fate imposes by taking refuge in God and following His commandments.  As a result of the working of eternal justice, there can be no escape from the consequences of our deeds. We become the product of our own past thinking and action. \nTherefore, we must think and act wisely at the present moment, using the scriptures as a guide.  "

In [ ]:
result['source_documents']

[Document(page_content='46                           International Gita Society \n \n \nIf due to ego you think: I shall not fight, your resolve is vain. \nBecause your own nature will compel you to fig ht. (18.59) O Ar-\njuna, you are controlled by your own nature -born Karmic impres-\nsions (or Samskara). Therefore, you shall do — even against your \nwill — what you do not wish to do out of delusion. (18.60) The Su-\npreme Lord, Krishna or God, abiding as the controller in the inner \npsyche of all beings, O Arjuna, causes them to evolve — or work \nout their Karma — like a puppet of Karma, mounted on a machine. \n(18.61)  \n  The Supreme Controller is the reflection of the Supreme \nSpirit in the body. The Supreme Lord organizes, controls, \nsupervises, and directs everything in the universe. Human being is \nlike a puppet of Karma mounted on a body which is the vehicle of \ntransmigration. The word evolve refers to working out one’s Karma. \nThus we become the puppet of our own Kar

In [ ]:
sources = ""
for count, source in enumerate(result['source_documents'],1):
  sources += "Source " + str(count) + "\n"
  sources += source.page_content + "\n"

print(sources)


Source 1
46                           International Gita Society 
 
 
If due to ego you think: I shall not fight, your resolve is vain. 
Because your own nature will compel you to fig ht. (18.59) O Ar-
juna, you are controlled by your own nature -born Karmic impres-
sions (or Samskara). Therefore, you shall do — even against your 
will — what you do not wish to do out of delusion. (18.60) The Su-
preme Lord, Krishna or God, abiding as the controller in the inner 
psyche of all beings, O Arjuna, causes them to evolve — or work 
out their Karma — like a puppet of Karma, mounted on a machine. 
(18.61)  
  The Supreme Controller is the reflection of the Supreme 
Spirit in the body. The Supreme Lord organizes, controls, 
supervises, and directs everything in the universe. Human being is 
like a puppet of Karma mounted on a body which is the vehicle of 
transmigration. The word evolve refers to working out one’s Karma. 
Thus we become the puppet of our own Karma created by our free 
will.  


In [ ]:
!zip -r /content/db.zip /content/db

  adding: content/db/ (stored 0%)
  adding: content/db/chroma-collections.parquet (deflated 47%)
  adding: content/db/chroma-embeddings.parquet (deflated 19%)
  adding: content/db/index/ (stored 0%)
  adding: content/db/index/uuid_to_id_ba87efb7-c97b-406f-8e6b-a90df3371b7b.pkl (deflated 40%)
  adding: content/db/index/id_to_uuid_ba87efb7-c97b-406f-8e6b-a90df3371b7b.pkl (deflated 31%)
  adding: content/db/index/index_ba87efb7-c97b-406f-8e6b-a90df3371b7b.bin (deflated 41%)
  adding: content/db/index/index_metadata_ba87efb7-c97b-406f-8e6b-a90df3371b7b.pkl (deflated 14%)


In [ ]:
from google.colab import files
files.download("/content/db.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#texts[:20]